# Call the Azure Function (Fabric + OAP + MPE)

This notebook calls the Hub Function to get a Service Principal token,
then prints the caller information and token metadata.

Notes:
- This is intended for Fabric with OAP enabled.
- To use the interactive user identity, run this in a notebook session (not a pipeline run).
- When using MPE, set the Function URL to the private endpoint FQDN.

Update the configuration values in the next cell before running.

In [ ]:
from notebookutils import mssparkutils
import requests

# Configuration
FUNC_APP_CLIENT_ID = "your-function-app-client-id"
FUNCTION_URL = "https://your-function-app.azurewebsites.net/api/GetSPToken"  # Use MPE private FQDN when enabled
TARGET_SCOPE = "https://management.azure.com/.default"

# Acquire token for the Function App (interactive user identity in notebook sessions)
audience = f"api://{FUNC_APP_CLIENT_ID}"
token = mssparkutils.credentials.getToken(audience)

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}
payload = {"targetScope": TARGET_SCOPE}

response = requests.post(FUNCTION_URL, headers=headers, json=payload, timeout=30)
print(f"Status: {response.status_code}")

if response.status_code == 200:
    body = response.json()
    caller = body.get("caller", {})
    print(f"Caller type: {caller.get('type', 'n/a')}")
    print(f"Caller display: {caller.get('display', 'n/a')}")
    print(f"Expires in: {body.get('expires_in')}")
else:
    print(response.text)